In [10]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.applications.vgg16 import VGG16

In [11]:
# Set the dimensions of your input images
img_width, img_height = 256, 256

# Set the paths to your dataset
train_data_dir = './data/garbage_training/training/'
validation_data_dir = './data/garbage_training/validation/'

# Set the number of training and validation samples
train_samples = 2000
validation_samples = 200

# Set the number of epochs and batch size
epochs = 10
batch_size = 8

# Load the VGG16 model without the top layer (fully connected layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [12]:
# Freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Create a new model
model = Sequential()

# Add the VGG16 base model to the new model
model.add(base_model)

# Add additional layers
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

In [13]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation for training images
train_datagen = ImageDataGenerator( rescale=1. / 255,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    rotation_range=30,  # Rotate images by 30 degrees
                                    width_shift_range=0.1,  # Shift images horizontally by 10%
                                    height_shift_range=0.1,  # Shift images vertically by 10%
                                    brightness_range=(0.8, 1.2),  # Adjust brightness between 0.8 and 1.2
                                    fill_mode='nearest')

# Rescale validation images
val_datagen = ImageDataGenerator(rescale=1. / 255)

In [14]:
# Load and generate training data
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# Load and generate validation data
validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')


Found 2390 images belonging to 6 classes.
Found 600 images belonging to 6 classes.


In [15]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_samples // batch_size)
    callbacks=[early_stop]


Epoch 1/10
62/62 [==============================] - 45s 689ms/step - loss: 2.2257 - accuracy: 0.4052 - val_loss: 1.0497 - val_accuracy: 0.6302
Epoch 2/10
62/62 [==============================] - 41s 656ms/step - loss: 1.2458 - accuracy: 0.4959 - val_loss: 1.0770 - val_accuracy: 0.5677
Epoch 3/10
62/62 [==============================] - 41s 658ms/step - loss: 1.1874 - accuracy: 0.4849 - val_loss: 0.9304 - val_accuracy: 0.6406
Epoch 4/10
62/62 [==============================] - 40s 652ms/step - loss: 1.0912 - accuracy: 0.5285 - val_loss: 0.8907 - val_accuracy: 0.6250
Epoch 5/10
62/62 [==============================] - 41s 655ms/step - loss: 1.0690 - accuracy: 0.5192 - val_loss: 0.7569 - val_accuracy: 0.7083
Epoch 6/10
62/62 [==============================] - 41s 658ms/step - loss: 0.9817 - accuracy: 0.5766 - val_loss: 0.8056 - val_accuracy: 0.6562
Epoch 7/10
62/62 [==============================] - 41s 657ms/step - loss: 1.0041 - accuracy: 0.5550 - val_loss: 0.8158 - val_accuracy: 0.6667

KeyboardInterrupt: 

In [ ]:
training_loss = history.history["loss"]
validation_loss = history.history["val_loss"]

# Get the training and validation accuracies from the history
training_accuracy = history.history["accuracy"]
validation_accuracy = history.history["val_accuracy"]

# Print the training and validation losses and accuracies for each epoch
for epoch in range(len(training_loss)):
    print(f"Epoch {epoch+1}:")
    print(f"Training Loss: {training_loss[epoch]:.4f} - Training Accuracy: {training_accuracy[epoch]:.4f}")
    print(f"Validation Loss: {validation_loss[epoch]:.4f} - Validation Accuracy: {validation_accuracy[epoch]:.4f}")
    print()

SyntaxError: unexpected EOF while parsing (4009565363.py, line 9)

In [ ]:
# Save the trained model
model.save('trained_model.h5')